In [3]:
def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r - 1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (
        int(maxArea[3] + 1 - maxArea[0] / abs(maxArea[1] - maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])

def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else:
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i - position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif (height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area = maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea

In [4]:
import numpy as np
import cv2
import  imutils
import pytesseract
#Read the image 

image=cv2.imread('car.jpeg')
#convert image from RGB to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Noise removal with iterative bilateral filter(removes noise while preserving edges)
gray = cv2.bilateralFilter(gray, 11, 17, 17)

# Find Edges of the filtered grayscale image
edged = cv2.Canny(gray, 170, 200)
# Find contours based on Edges
_, thresh = cv2.threshold(edged, 10, 255, cv2.THRESH_BINARY)
(cnts,_) = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30] #sort contours based on their area keeping minimum required area as '30' (anything smaller than this will not be considered)
ff = np.zeros((edged.shape[0], edged.shape[1]), 'uint8')
cv2.drawContours(ff, cnts, -1, 1, 15)    
ff_mask = np.zeros((edged.shape[0] + 2, edged.shape[1] + 2), 'uint8')
cv2.floodFill(ff, ff_mask, (int(edged.shape[1] / 2), int(edged.shape[0] / 2)), 1)    
   
rect = maxRect(ff)
rectangle = [min(rect[0], rect[2]), max(rect[0], rect[2]), min(rect[1], rect[3]), max(rect[1], rect[3])]
img_crop = image[rectangle[0]:rectangle[1], rectangle[2]:rectangle[3]]


# Convert to gray
img = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
 # Apply dilation and erosion to remove some noise
kernel = np.ones((1, 1), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)
img = cv2.erode(img, kernel, iterations=1)

    # Write the image after apply opencv to do some ...
cv2.imwrite("thresimg.png", img)
    # Recognize text with tesseract for python
result = pytesseract.image_to_string(cv2.imread("thresimg.png"))




In [5]:
print(result)

MH12DE 1433]
